## Imports

In [1]:
import findspark

findspark.init('C:/spark')

In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.ml.regression import LinearRegression

## Create a Session

In [4]:
spark = SparkSession.builder.appName('linreg').getOrCreate()

## Read Data

In [5]:
data = spark.read.csv('../../data/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

## MLlib Imports

In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

## Data Transformation

In [9]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [10]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
                                       'Time on App',
                                       'Time on Website',
                                       'Length of Membership'],
                            outputCol='features')

In [11]:
output = assembler.transform(data)

In [12]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [13]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [14]:
final_data = output.select('features', 'Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

## Train Test Split

In [15]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [16]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                356|
|   mean|  497.9333158161641|
| stddev|  78.51685371164146|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [17]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                144|
|   mean|   502.727490963479|
| stddev|  81.43089840422464|
|    min|  275.9184206503857|
|    max|  725.5848140556806|
+-------+-------------------+



## Model

In [18]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [19]:
lr_model = lr.fit(train_data)

In [20]:
test_results = lr_model.evaluate(test_data)

In [21]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -0.3569757837822749|
|  10.833824454959881|
|   6.590250591167205|
|  -21.36106481411059|
|  -7.270657991970211|
|  22.272758488969203|
|-0.08447106929656911|
|  -5.009808562432511|
| -17.240596113460697|
| -1.9829071249929484|
|  17.481318801487134|
|   6.822886397150512|
|  -5.707469694808367|
| -11.469137880982089|
| -2.3643289192313546|
|  -9.247471238296555|
|  -2.464586571236623|
|  -9.347589167122749|
|   -13.9136632996765|
|  -5.657849912570896|
+--------------------+
only showing top 20 rows



In [22]:
test_results.rootMeanSquaredError

9.590288860248643

In [23]:
test_results.r2

0.9860327348156819

## Unlabeled

In [24]:
unlabeled_data = test_data.select('features')

In [25]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.5743636841713...|
|[30.7377203726281...|
|[30.9716756438877...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.2834474760581...|
|[31.3895854806643...|
|[31.5171218025062...|
|[31.5702008293202...|
|[31.5761319713222...|
|[31.6098395733896...|
|[31.6548096756927...|
|[31.7242025238451...|
|[31.8093003166791...|
|[31.8186165667690...|
|[31.8279790554652...|
|[31.8530748017465...|
|[31.8854062999117...|
|[31.9365486184489...|
|[31.9453957483445...|
+--------------------+
only showing top 20 rows



In [26]:
predictions = lr_model.transform(unlabeled_data)

In [27]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.5743636841713...|442.42138954184793|
|[30.7377203726281...|   450.94691774127|
|[30.9716756438877...| 488.0483591657255|
|[31.1239743499119...|508.30811865387636|
|[31.1280900496166...| 564.5233447390249|
|[31.2834474760581...| 569.5083309366983|
|[31.3895854806643...|410.15408212927946|
|[31.5171218025062...| 280.9282292128182|
|[31.5702008293202...| 563.1860882548656|
|[31.5761319713222...| 543.2094911143213|
|[31.6098395733896...|  427.064230849621|
|[31.6548096756927...|  468.440537330398|
|[31.7242025238451...|509.09535698276886|
|[31.8093003166791...| 548.2410372438233|
|[31.8186165667690...|  448.783002289367|
|[31.8279790554652...| 449.2502187852381|
|[31.8530748017465...| 461.7497100335886|
|[31.8854062999117...|399.45086213959826|
|[31.9365486184489...| 441.1130481950047|
|[31.9453957483445...| 662.6777738502228|
+--------------------+------------